# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Widget-parametrization" data-toc-modified-id="Widget-parametrization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Widget parametrization</a></div>

# Widget parametrization 

In [78]:
from __future__ import print_function
import requests
import json
from pprint import pprint
from IPython.display import display, Markdown
import pandas as pd
from ipywidgets import DatePicker, interactive, Textarea, Layout, Dropdown
import ipywidgets as widgets
from datetime import datetime
from vega import Vega

We are going to use this dataset `20cc5eca-8c63-4c41-8e8e-134dcf1e6d76` (Active Fires VIIRS) as an example for widget creation.  
`http://api.resourcewatch.org/fields/:dataset_id`  
Field information to be able to build the dynamic query: 

```json  
{
"tableName": "vnp14imgtdl_nrt_global_7d",
"fields": {
"cartodb_id": {
"type": "number"
},
"the_geom": {
"type": "geometry"
},
"the_geom_webmercator": {
"type": "geometry"
},
"latitude": {
"type": "number"
},
"longitude": {
"type": "number"
},
"bright_ti4": {
"type": "number"
},
"scan": {
"type": "number"
},
"track": {
"type": "number"
},
"acq_date": {
"type": "date"
},
"acq_time": {
"type": "string"
},
"satellite": {
"type": "string"
},
"confidence": {
"type": "string"
},
"version": {
"type": "string"
},
"bright_ti5": {
"type": "number"
},
"frp": {
"type": "number"
},
"daynight": {
"type": "string"
},
"_feature_count": {
"type": "number"
}
}
}
```



With this we can define the sql query that we want to use. It is very important, if you are parametrizing the queries, you will need to make sure you keep the same columns and their aliases  
```sql
SELECT julian_day as x, count(julian_day) 
FROM index_e663eb0904de4f39b87135c6c2ed10b5 
WHERE confidence > 50 and year > {year_begin} and julian_day >{dayyear_begin} and year < {year_end}  and julian_day < {dayyear_end}
```

We are going to define two simple widgets in vega 2.6 and add them to the api:
* Bar: 
    * Data layout:
 
| date | value |  
|---|---|  
| | |  
| | |  
* Line:
    * Data layout:
  
|x|y|  
|---|---|  
| | |  
| | |  

* Pie:
    * Data layout: 
 
|category|value|  
|---|---|  
|||  
|||  






In [79]:
api_baseurl = 'https://api.resourcewatch.org/v1'

def julian_day(date):
    first_dayyear=datetime(date.year,1,1)
    return abs(date - first_dayyear).days + 1


def visualization(widget, base_queryUrl, sql, date_begin, date_end, geostore=None):
    
    api_widgetapi = api_baseurl + '/widget/{0}'.format(widget)
    payload = {"queryUrl":base_queryUrl.format(sql.format(date_begin=date_begin.isoformat(), date_end=date_end.isoformat()))}
    if geostore!=None:
        payload.update({'geostore':geostore})
    r = requests.get(api_widgetapi, params=payload)
    if r.status_code != 200:
        print(r.json())
    else:
        spec = json.dumps(r.json()['data']['attributes']['widgetConfig'])
        display(Vega(json.loads(spec)));
        display(Markdown("#### Call:  "));
        display(Markdown(">>```  {0}  ```".format(r.url)));
        display(Markdown("#### Response:  "));
        display(Markdown(""">> ```json  
        {0}  ```""".format((spec))));

In [81]:
pos=Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')
options= {
    'widgets':Dropdown(options={'line': 'a5796158-1342-4b1a-8a70-914d630738e2','bar': '19143822-3b11-407a-b3f7-7e98f6a1a024'}, description='Widget type:'),
    'base_queryUrl':Textarea(value=api_baseurl+"/query/20cc5eca-8c63-4c41-8e8e-134dcf1e6d76?sql={0}", description='Url to query:',layout=pos),
    'sql': Textarea(value="SELECT acq_date as x, count(acq_date) as y FROM vnp14imgtdl_nrt_global_7d WHERE acq_date >='{date_begin}' and acq_date < '{date_end}' group by acq_date order by acq_date asc", description='Query:',layout=pos),    
    'other_params':{'geostore'},
    'date_end': DatePicker(description='Date end', disabled=False, value=datetime.today()),
    'date_begin': DatePicker(description='Date begin', disabled=False, value=datetime(2017,10,18)),
    'geostore': Dropdown(options={'Afghanistan': '37b397a9a2860beb5c43a1f418d3c3d3', 'Brazil': 'd653e4fc0ed07a65b9db9b13477566fe', 'Custom area':'ae52478dc8ffe23f7fe175611c20b684'}, description='Geostore')  
}

display(interactive(visualization, widget=options['widgets'],base_queryUrl=options['base_queryUrl'], sql=options['sql'], date_begin=options['date_begin'], date_end=options['date_end'], geostore=options['geostore']));